In [1]:
import numpy as np
import pandas as pd

In [5]:
class DataGen():
    
    def __init__(self,win):
        
        """"""
        for j in range(1,2):
            combine_buffer=pd.DataFrame()
            for i in range(1):

                self.raw_data=pd.read_excel('RawData/P_{1}_S_{0}.xlsx'.format(i,j))
                print(len(self.raw_data))
                len(self.raw_data)
                

                self.raw_data=self.series_to_supervised(np.asarray(self.raw_data),win)
                
                if(i==0):
                    self.raw_data['Label']='Low_Stress'

                if(i==1):

                    self.raw_data['Label']='Medium_Stress'
                if(i==2):

                    self.raw_data['Label']='High_Stress'

                combine_buffer=pd.concat([combine_buffer,self.raw_data])

            combine_buffer.to_csv('Datasets/P_{0}_win_{1}.csv'.format(j,win),index=False)
        
        
        
        
    
    def series_to_supervised(self,data, n_in=1, dropnan=True):




        n_vars = 1 if type(data) is list else data.shape[1]
        df = pd.DataFrame(data)
        cols, names = list(), list()
        # input sequence (t-n, ... t-1)
        for i in range(n_in, 0, -1):
            cols.append(df.shift(i))
            names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
        # put it all together
        agg = pd.concat(cols, axis=1)
        agg.columns = names
        # drop rows with NaN values
        if dropnan:
            agg.dropna(inplace=True)

        return agg

In [6]:
ins=DataGen(1000)

30126


In [7]:
len(ins.raw_data)

29126

In [ ]:
P1=pd.read_csv('Datasets/P_1.csv')


In [ ]:
P2=pd.read_csv('Datasets/P_8_Win_1000.csv')

In [ ]:
import tensorflow as tf

from tensorflow.contrib import rnn
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm


In [ ]:
def LSTM_RNN_vx(X_train,Y_train,X_test,Y_test,units,epoch,batch_size,inputshape,lr=0.01):
    #Modeling RNN-LSTM
        model=tf.keras.Sequential()
        model.add(tf.keras.layers.LSTM (units,activation = 'sigmoid',input_shape =inputshape, return_sequences = False))
        
        rms=tf.keras.optimizers.RMSprop(lr=lr)
        model.compile(loss = 'categorical_crossentropy', optimizer = rms,metrics=['acc'])
        hist=model.fit(X_train,Y_train,epochs=epoch,batch_size=batch_size,shuffle=True,validation_data=(X_test,Y_test))
        Y_pred=model.predict(X_test)
        return hist,Y_pred,model

In [ ]:
L=P1['Label']

In [ ]:
L2=P2['Label']

In [ ]:
F=np.asarray(P1.drop('Label',axis=1))

In [ ]:
F2=np.asarray(P2.drop('Label',axis=1))

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.fit(L)
encoded_train_label = encoder.transform(L)

In [ ]:
encoder.fit(L2)
encoded_train_label_2 = encoder.transform(L2)

In [ ]:
F=F.reshape(F.shape[0],F.shape[1],1)

In [ ]:
F2=F2.reshape(F2.shape[0],F2.shape[1],1)

In [ ]:
from keras.utils import np_utils
dummy_y = np_utils.to_categorical(encoded_train_label)

In [ ]:
dummy_y2 = np_utils.to_categorical(encoded_train_label_2)

In [ ]:
hist,Y_pred,model=LSTM_RNN_vx(X_train=F,Y_train=dummy_y,units=3,epoch=10,batch_size=500,inputshape=(1000,1),X_test=F2,Y_test=dummy_y2)